In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Embedding, Dense, Dropout, GlobalMaxPooling1D, Concatenate, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import re
import json
import joblib
import matplotlib.pyplot as plt
# ----------------------------------------
# Data Preprocessing, Tokenization, Vocabulary Creation, Padding, and Standardization

# Set the output directory
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/LSTM-SA_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the text preprocessing function
def preprocess_text(text):
    # Convert to lower case and remove commas and periods
    text = text.lower()
    text = re.sub(r'[,.:]', '', text)
    return text

# Read data from CSV
data_path = 'E:/NC/FTD-Data.csv'
data = pd.read_csv(data_path, encoding='gbk')

# Extract numerical features and target variables
X_numeric = data.iloc[:, :18].values.astype(np.float32)
y = data.iloc[:, -2:].values.astype(np.float32)
concrete_types = data['CT'].values  # For stratification

# Extract text features and preprocess (columns 18 to 21)
X_text = data.iloc[:, 18:22].astype(str)
X_text = X_text.applymap(preprocess_text)

# Tokenize text using NLTK's word_tokenize
tokenized_texts = X_text.applymap(word_tokenize)

# Create vocabulary and build word index (0 is reserved for padding)
vocab = set([word for sublist in tokenized_texts.values.flatten() for word in sublist])
word_index = {word: i + 1 for i, word in enumerate(vocab)}
vocab_size = len(word_index)
print("Vocabulary size:", vocab_size)

# Save the word index as CSV
vocab_df = pd.DataFrame(list(word_index.items()), columns=['Word', 'Index'])
vocab_df.to_csv(os.path.join(output_dir, 'vocab_index.csv'), index=False)

# Save the word index as JSON
with open(os.path.join(output_dir, 'vocab_index.json'), 'w', encoding='utf-8') as json_file:
    json.dump(word_index, json_file, ensure_ascii=False, indent=4)

# Convert tokenized texts to sequences using the word index
sequences = tokenized_texts.applymap(lambda x: [word_index[word] for word in x])

# Determine the maximum length for each text column and save to CSV
max_lengths = [max(len(seq) for seq in sequences.iloc[:, i]) for i in range(4)]
max_lengths_df = pd.DataFrame({'Column': X_text.columns, 'Max_Length': max_lengths})
max_lengths_df.to_csv(os.path.join(output_dir, 'max_lengths.csv'), index=False)
print("Max lengths of each text column:", max_lengths)

# Split the dataset (Stratified 80:20 split based on concrete types)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_numeric, concrete_types):
    X_train_numeric, X_test_numeric = X_numeric[train_index], X_numeric[test_index]
    X_train_text, X_test_text = X_text.iloc[train_index], X_text.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    concrete_types_train, concrete_types_test = concrete_types[train_index], concrete_types[test_index]

# Convert text to sequences and perform padding for each column
X_train_text_padded = []
X_test_text_padded = []
for col in X_text.columns:
    train_texts = [word_tokenize(text) for text in X_train_text[col]]
    test_texts = [word_tokenize(text) for text in X_test_text[col]]
    train_sequences = [[word_index[word] for word in text if word in word_index] for text in train_texts]
    test_sequences = [[word_index[word] for word in text if word in word_index] for text in test_texts]
    max_len = max_lengths[X_text.columns.get_loc(col)]
    X_train_text_padded.append(pad_sequences(train_sequences, maxlen=max_len, padding='post'))
    X_test_text_padded.append(pad_sequences(test_sequences, maxlen=max_len, padding='post'))

# Standardize numerical features and save the scaler
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)
scaler_path = os.path.join(output_dir, 'scaler.pkl')
joblib.dump(scaler, scaler_path)

# Print shape of padded text inputs
for column_name, padded_array in zip(X_text.columns, X_train_text_padded):
    print(f"Shape of {column_name}: {padded_array.shape}")

# ----------------------------------------
# Model Definition, Training, and Saving Training History

# Define a Single-Head Self-Attention layer
class SingleHeadAttention(Layer):
    def __init__(self, embed_dim, proj_dim=None, output_dim=None, return_attention_weights=False, **kwargs):
        super(SingleHeadAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.proj_dim = proj_dim if proj_dim is not None else embed_dim
        self.output_dim = output_dim if output_dim is not None else embed_dim
        self.return_attention_weights = return_attention_weights

        self.q_dense = Dense(self.proj_dim, use_bias=False, name="query_projection")
        self.k_dense = Dense(self.proj_dim, use_bias=False, name="key_projection")
        self.v_dense = Dense(self.proj_dim, use_bias=False, name="value_projection")
        self.output_dense = Dense(self.output_dim, use_bias=False, name="output_dense")
        self.supports_masking = True

    def call(self, inputs, mask=None):
        q = self.q_dense(inputs)
        k = self.k_dense(inputs)
        v = self.v_dense(inputs)
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        if mask is not None:
            mask = tf.cast(mask, tf.float32)
            mask_expanded = tf.expand_dims(mask, axis=-1)
            matmul_mask = tf.matmul(mask_expanded, mask_expanded, transpose_b=True)
            scaled_attention_logits += (1 - matmul_mask) * -1e9
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        attention_output = tf.matmul(attention_weights, v)
        output = self.output_dense(attention_output)
        if self.return_attention_weights:
            return output, attention_weights
        return output

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'proj_dim': self.proj_dim,
            'output_dim': self.output_dim,
            'return_attention_weights': self.return_attention_weights,
            'supports_masking': self.supports_masking
        })
        return config

# Create model function using single-head self-attention
def create_model(max_lengths, vocab_size, embedding_dim, proj_dim, output_dim, include_attention_weights=False):
    inputs = []
    main_outputs = []
    attention_weights_outputs = []
    for i, max_len in enumerate(max_lengths):
        input_layer = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True, trainable=True)(input_layer)
        lstm_output = Bidirectional(LSTM(16, return_sequences=True))(embedding_layer)
        attention_layer = SingleHeadAttention(embed_dim=embedding_dim, proj_dim=proj_dim, output_dim=output_dim,
                                               return_attention_weights=include_attention_weights)
        if include_attention_weights:
            attention_output, attention_weights = attention_layer(lstm_output)
            attention_weights_outputs.append(attention_weights)
        else:
            attention_output = attention_layer(lstm_output)
        pooled_output = GlobalMaxPooling1D()(attention_output)
        main_outputs.append(pooled_output)
        inputs.append(input_layer)
    numeric_input = Input(shape=(18,), dtype='float32', name="numeric_input")
    inputs.append(numeric_input)
    combined_features = Concatenate()([*main_outputs, numeric_input])
    x = Dense(128, activation='relu', kernel_initializer='he_normal')(combined_features)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    final_output = Dense(2)(x)
    if include_attention_weights:
        model = Model(inputs=inputs, outputs=[final_output] + attention_weights_outputs)
    else:
        model = Model(inputs=inputs, outputs=[final_output])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
    return model

# Training phase: Use 5-fold stratified cross-validation, save training history (CSV) and models.
loss_history = []  # Record training history per fold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_numeric_scaled, concrete_types_train)):
    print(f"Training fold {fold+1}/5")
    # Prepare numeric training and validation data
    X_train_fold_numeric = X_train_numeric_scaled[train_idx]
    X_val_fold_numeric = X_train_numeric_scaled[val_idx]
    y_train_fold = y_train[train_idx]
    y_val_fold = y_train[val_idx]

    # Create model
    model = create_model(max_lengths, vocab_size, embedding_dim=64, proj_dim=32, output_dim=32, include_attention_weights=False)

    # Prepare text inputs (order: text inputs then numeric input)
    train_inputs = [X_train_text_padded[i][train_idx] for i in range(len(max_lengths))]
    val_inputs = [X_train_text_padded[i][val_idx] for i in range(len(max_lengths))]
    train_inputs.append(X_train_fold_numeric)
    val_inputs.append(X_val_fold_numeric)

    # Train the model
    history = model.fit(train_inputs, y_train_fold,
                        validation_data=(val_inputs, y_val_fold),
                        epochs=3, batch_size=64, verbose=1)
    # Save training history as CSV
    df_history = pd.DataFrame(history.history)
    history_csv_path = os.path.join(output_dir, f'fold_{fold+1}_history.csv')
    df_history.to_csv(history_csv_path, index=False)
    print(f"Fold {fold+1} training history saved to: {history_csv_path}")
    loss_history.append(history.history)

    # Save model
    model_save_path = os.path.join(output_dir, f'model_fold_{fold+1}.h5')
    model.save(model_save_path)
    print(f"Fold {fold+1} model saved to: {model_save_path}")

# ----------------------------------------
# Testing phase: For each fold, load the corresponding model, compute evaluation metrics, and save detailed CSVs and a summary CSV.
test_metrics = {
    'Fold': [],
    'Target': [],
    'MAE': [],
    'MSE': [],
    'RMSE': [],
    'R2': []
}

# Process test metrics for each fold
for fold in range(1, 6):
    print(f"Processing test metrics for fold {fold}")
    # Load the model (using load_weights via a new model instance)
    model = create_model(max_lengths, vocab_size, embedding_dim=64, proj_dim=32, output_dim=32, include_attention_weights=False)
    model.load_weights(os.path.join(output_dir, f'model_fold_{fold}.h5'))
    
    # Prepare test inputs (text + numeric)
    test_inputs = [X_test_text_padded[i] for i in range(len(max_lengths))]
    test_inputs.append(X_test_numeric_scaled)
    
    predictions = model.predict(test_inputs)
    # If predictions is a list, use the first element as the prediction
    pred_output = predictions[0] if isinstance(predictions, list) else predictions
    
    # Compute metrics for each target (assuming 2 targets)
    for target in range(2):
        mae = mean_absolute_error(y_test[:, target], pred_output[:, target])
        mse = mean_squared_error(y_test[:, target], pred_output[:, target])
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test[:, target], pred_output[:, target])
        
        test_metrics['Fold'].append(fold)
        test_metrics['Target'].append(f"target{target+1}")
        test_metrics['MAE'].append(mae)
        test_metrics['MSE'].append(mse)
        test_metrics['RMSE'].append(rmse)
        test_metrics['R2'].append(r2)
        
        print(f"Fold {fold} - Target {target+1}: MAE={mae:.4f}, MSE={mse:.4f}, RMSE={rmse:.4f}, R2={r2:.4f}")
        
        # Save the predictions along with actual values and absolute error
        df_pred = pd.DataFrame({
            'Category': concrete_types_test,
            'Actual Value': y_test[:, target],
            'Predicted Value': pred_output[:, target],
            'Absolute Error': np.abs(y_test[:, target] - pred_output[:, target])
        })
        pred_csv_path = os.path.join(output_dir, f'Predictions_Fold_{fold}_Target_{target+1}.csv')
        df_pred.to_csv(pred_csv_path, index=False)
        print(f"Fold {fold} - Target {target+1} predictions saved to: {pred_csv_path}")

# Save overall test metrics as CSV
df_metrics = pd.DataFrame(test_metrics)
metrics_csv_path = os.path.join(output_dir, 'Test_Metrics.csv')
df_metrics.to_csv(metrics_csv_path, index=False)
print(f"All test metrics saved to: {metrics_csv_path}")

# Compute average test metrics per target and save summary CSV
avg_metrics = df_metrics.groupby('Target').mean().reset_index()
avg_csv_path = os.path.join(output_dir, 'Average_Test_Metrics.csv')
avg_metrics.to_csv(avg_csv_path, index=False)
print("Average test metrics:")
print(avg_metrics)
print(f"Average test metrics saved to: {avg_csv_path}")

print("All training histories and test metrics have been saved as CSV files.")

# ----------------------------------------
# Plot training and validation loss curves for each fold
for i, history in enumerate(loss_history):
    plt.figure(figsize=(8, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curves for Fold {i+1}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plot_save_path = os.path.join(output_dir, f'Loss_Curve_Fold_{i+1}.svg')
    plt.savefig(plot_save_path)
    plt.close()
    print(f"Fold {i+1} loss curve saved to: {plot_save_path}")
